# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [1]:
import pandas as pd
import pymongo

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
#connecter avec mongodb
client = pymongo.MongoClient("mongo:27017")
#créer un database "exercices" et un collection "youtube"
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [4]:
df_youtube = df_youtube.dropna()

In [5]:
#importer le fichier json à dataframe
import json

with open('./data/US_category_id.json') as data_file:    
    data = json.load(data_file)  

df_categorie = pd.json_normalize(data, 'items')
df_categorie.head(5)

,kind,etag,id,snippet.channelId,snippet.title,snippet.assignable
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


In [6]:
#nettoyer le dataframe de catégorie
df_categorie=df_categorie.rename(columns={'id':'category_id'})
df_categorie=df_categorie.rename(columns={'snippet.channelId':'snippet_channelId'})
df_categorie=df_categorie.rename(columns={'snippet.title':'snippet_title'})
df_categorie=df_categorie.rename(columns={'snippet.assignable':'snippet_assignable'})
df_categorie["category_id"] = pd.to_numeric(df_categorie["category_id"])
df_categorie.head(5)

,kind,etag,category_id,snippet_channelId,snippet_title,snippet_assignable
0,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,youtube#videoCategory,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


In [7]:
df_categorie=df_categorie.drop(['kind'], axis=1)
df_categorie.head(5)

,etag,category_id,snippet_channelId,snippet_title,snippet_assignable
0,"""m2yskBQFythfE4irbTIeOgYYfBU/Xy1mB4_yLrHy_BmKm...",1,UCBR8-60-B28hp2BmDPdntcQ,Film & Animation,True
1,"""m2yskBQFythfE4irbTIeOgYYfBU/UZ1oLIIz2dxIhO45Z...",2,UCBR8-60-B28hp2BmDPdntcQ,Autos & Vehicles,True
2,"""m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxb...",10,UCBR8-60-B28hp2BmDPdntcQ,Music,True
3,"""m2yskBQFythfE4irbTIeOgYYfBU/HwXKamM1Q20q9BN-o...",15,UCBR8-60-B28hp2BmDPdntcQ,Pets & Animals,True
4,"""m2yskBQFythfE4irbTIeOgYYfBU/9GQMSRjrZdHeb1OEM...",17,UCBR8-60-B28hp2BmDPdntcQ,Sports,True


### Importer les données

In [8]:
#merger le dataframe de catégorie pour récupérer le nom des catégories.
df = pd.merge(df_youtube,df_categorie, on='category_id',how='left')

df.head(5)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date,etag,snippet_channelId,snippet_title,snippet_assignable
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/UVB9oxX2Bvqa_w_y3...",UCBR8-60-B28hp2BmDPdntcQ,Entertainment,True
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
2,cLdxuaxaQwc,My Response,PewDiePie,22,[none],5845909,576597,39774,170708,https://i.ytimg.com/vi/cLdxuaxaQwc/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/xId8RX7vRN8rqkbYZ...",UCBR8-60-B28hp2BmDPdntcQ,People & Blogs,True
3,WYYvHb03Eog,Apple iPhone X first look,The Verge,28,apple iphone x hands on|Apple iPhone X|iPhone ...,2642103,24975,4542,12829,https://i.ytimg.com/vi/WYYvHb03Eog/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBc...",UCBR8-60-B28hp2BmDPdntcQ,Science & Technology,True
4,sjlHnJvXdQs,iPhone X (parody),jacksfilms,23,jacksfilms|parody|parodies|iphone|iphone x|iph...,1168130,96666,568,6666,https://i.ytimg.com/vi/sjlHnJvXdQs/default.jpg,13.09,"""m2yskBQFythfE4irbTIeOgYYfBU/G9LHzQmx44rX2S5ya...",UCBR8-60-B28hp2BmDPdntcQ,Comedy,True


In [9]:
#importer les donnée de fichier USvideos.csv
collection.insert_many(df.to_dict('records'))

## Question 1  

In [10]:
#1) Récupérer toutes les vidéos de la chaîne Apple.
chaine_apple = collection.find( { "channel_title":  {"$all" :["Apple"] } } )
list(chaine_apple)

[{'_id': ObjectId('61f60c7e037066ab23efdd62'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone',
  'views': 7860119,
  'likes': 185853,
  'dislikes': 26679,
  'comment_total': 0,
  'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg',
  'date': 13.09,
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/w5HjcTD82G_XA3xBctS30zS-JpQ"',
  'snippet_channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'snippet_title': 'Science & Technology',
  'snippet_assignable': True},
 {'_id': ObjectId('61f60c7e037066ab23efde2c'),
  'video_id': 'K4wEI5zhHB0',
  'title': 'iPhone X — Introducing iPhone X — Apple',
  'channel_title': 'Apple',
  'category_id': 28,
  'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|sm

## Question 2

In [11]:
#2) Compter le nombre de catégories différentes
categorie=collection.distinct('snippet_title')
print("le list de catégories différentesest :", categorie)
print("le nombre de catégories différentesest :", len(categorie))

le list de catégories différentesest : ['Autos & Vehicles', 'Comedy', 'Education', 'Entertainment', 'Film & Animation', 'Gaming', 'Howto & Style', 'Music', 'News & Politics', 'Nonprofits & Activism', 'People & Blogs', 'Pets & Animals', 'Science & Technology', 'Shows', 'Sports', 'Travel & Events']
le nombre de catégories différentesest : 16


## Question 3

In [12]:
#3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
tags=collection.distinct('tags')
print(tags)

['#DNN|DNN|MILO|TRUMP|Alex|Jones|Protestors|ANTIFA|Pepper|Mace|Police|Arrests|NFL|BoycottTheNFL|Boycott the NFL|NFL Ghost Town|Empty stadiums|Season tickets|ratings|owners|players|kneel|kneeling', '#JumanjiMovie|Jumanji trailer|Sony Pictures|Jumanji 2|Jumanji: Welcome To The Jungle|The Rock|Dwayne Johnson|Kevin Hart|Karen Gillan|Jack Black|Nick Jonas|Jumanji sequel|Jumanji remake|Jumanji reboot|new Jumanji|Jumanji video game|official trailer|Jumanji official trailer|Jumanji: Welcome To The Jungle Official Trailer|new Jumanji trailer|The Rock and Kevin Hart|Jumanji 2 Welcome To The Jungle|movie trailer|HD|Jumanji|bobby cannavale|Robin Williams', '#LemonPepperShawties|Chicken wings|The Film Culture|Film Culture|Black Hollywood|Issa Rae|Yvonne Orji|Insecure|Lil Rel Howery|Jidenna|Janelle Monae|ATL|Atlanta|Black Film|Black Films|Black Movies|Luke James|BET|New Edition|The New Edition Story|Black People|Lit|Party|Instagram|IG|Social Media|The Shade Room', "'stephen colbert'|stars|awards|tel

## Question 4

In [41]:
#4) Récupérer les vidéos les plus vues.
view_max=collection.find().sort([("views", -1)]).limit(20)
print("les 20 vidéos les plus vues : ")
list(view_max)

les 20 vidéos les plus vues : 


[{'_id': ObjectId('61f60c7e037066ab23efe4d4'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09,
  'etag': '"m2yskBQFythfE4irbTIeOgYYfBU/nqRIq97-xe5XRZTxbknKFVe5Lmg"',
  'snippet_channelId': 'UCBR8-60-B28hp2BmDPdntcQ',
  'snippet_title': 'Music',
  'snippet_assignable': True},
 {'_id': ObjectId('61f6c7a63f8293cf9c998858'),
  'video_id': 'MBdVXkSdhwU',
  'title': "BTS (방탄소년단) 'DNA' Official MV",
  'channel_title': 'ibighit',
  'category_id': 10,
  'tags': 'BIGHIT|빅히트|방탄소년단|BTS|BANGTAN|방탄',
  'views': 41500672,
  'likes': 2010366,
  'dislikes': 78076,
  'comment_total': 736179,
  'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg',
  'date': 22.09,
  'etag': '"m2yskBQFythfE4irbTIeOgY

## Question 5

In [44]:
#5) Compter le nombre moyen de vues en fonction de la catégorie.
Moy_views=collection.aggregate([{'$group':{'_id':'$snippet_title','Moyenne de vues':{'$avg':"$views"}}}])
list(Moy_views)

[{'_id': 'Travel & Events', 'Moyenne de vues': 464041.0833333333},
 {'_id': 'People & Blogs', 'Moyenne de vues': 971532.7573696146},
 {'_id': 'Entertainment', 'Moyenne de vues': 1154868.4397251718},
 {'_id': 'Howto & Style', 'Moyenne de vues': 537665.1806674339},
 {'_id': 'Education', 'Moyenne de vues': 547582.4700598803},
 {'_id': 'Gaming', 'Moyenne de vues': 681081.8170731707},
 {'_id': 'Autos & Vehicles', 'Moyenne de vues': 607693.8706896552},
 {'_id': 'Film & Animation', 'Moyenne de vues': 1039472.6560846561},
 {'_id': 'Music', 'Moyenne de vues': 1176553.612},
 {'_id': 'News & Politics', 'Moyenne de vues': 540955.5569823436},
 {'_id': 'Pets & Animals', 'Moyenne de vues': 651404.3879310344},
 {'_id': 'Sports', 'Moyenne de vues': 728434.3243902439},
 {'_id': 'Comedy', 'Moyenne de vues': 1240073.294039735},
 {'_id': 'Nonprofits & Activism', 'Moyenne de vues': 1110334.2142857143},
 {'_id': 'Shows', 'Moyenne de vues': 8492.5},
 {'_id': 'Science & Technology', 'Moyenne de vues': 924730.5

## Question 6 

In [43]:
#6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.
Moy_likes=collection.aggregate([{'$group':{'_id':'$channel_title','Moyenne de likes':{'$avg':"$likes"}}},{"$sort": {"Moy_likes": -1}},{"$limit": 20}])
print("Les 20 chaines Youtube avec la plus grande moyenne de likes sont :")
list(Moy_likes)

Les 20 chaines Youtube avec la plus grande moyenne de likes sont :


[{'_id': 'Kevin The Tech Ninja', 'Moyenne de likes': 370.5},
 {'_id': 'FOXSportsMidwest', 'Moyenne de likes': 1592.25},
 {'_id': 'Joanna Hausmann', 'Moyenne de likes': 6632.0},
 {'_id': 'Flossy Carter', 'Moyenne de likes': 4331.5},
 {'_id': 'CalvinHarrisVEVO', 'Moyenne de likes': 148194.5},
 {'_id': 'Reach Records', 'Moyenne de likes': 3333.3636363636365},
 {'_id': 'Made by Google', 'Moyenne de likes': 0.0},
 {'_id': 'OlanRogers', 'Moyenne de likes': 16206.0},
 {'_id': 'Strictly Dumpling', 'Moyenne de likes': 8607.545454545454},
 {'_id': 'HaileeSteinfeldVEVO', 'Moyenne de likes': 38607.75},
 {'_id': 'CartoonHangover', 'Moyenne de likes': 4044.6},
 {'_id': 'Chris Cohoon', 'Moyenne de likes': 31525.8},
 {'_id': 'CarlyPearceVEVO', 'Moyenne de likes': 162.0},
 {'_id': 'consumer', 'Moyenne de likes': 6373.909090909091},
 {'_id': 'Steve Mould', 'Moyenne de likes': 6048.25},
 {'_id': 'CelebNews17', 'Moyenne de likes': 1477.0},
 {'_id': 'Rad Universe', 'Moyenne de likes': 11.0},
 {'_id': 'Mrwh